# Diccionario Inverso con Python

In [1]:
import xlrd
import networkx as nx
import nltk
import spacy
import string
import os

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 60.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
nlp = spacy.load("es_core_news_sm")

In [5]:
xlsx = "/content/drive/MyDrive/PLN_UADY/NAP.xls"

In [6]:
# Funcion para generar nuestros grafos
def build_graph(xlsx):
  data = xlrd.open_workbook(xlsx)
  hoja = data.sheet_by_index(0)
  GrafoFrec = nx.Graph()
  GrafoTiempo = nx.Graph()
  GrafoAsoc = nx.Graph()
  num_filas = hoja.nrows
  estimulo, lema = "", ""
  frecuencia, tiempo, asociado = 0, 0, 0
  for fila in range(0,num_filas):
    celda = hoja.cell(fila,0)
    cadena = celda.value
    if cadena == "--PALABRAS--":
      continue
    elif cadena == "======":
      estimulo = hoja.cell(fila+1,4).value
    elif cadena == "" or cadena == "=":
      continue
    elif hoja.cell(fila,2).value =="":
      continue
    else:
      frecuencia = hoja.cell(fila,1).value
      tiempo = hoja.cell(fila,2).value
      asociado = hoja.cell(fila,3).value
      palabra = hoja.cell(fila,4).value
      estimulo = estimulo.strip()
      lema = str(palabra)
      lema = lema.strip()
      GrafoFrec.add_edge(estimulo, lema, weight=1/float(frecuencia))
      GrafoTiempo.add_edge(estimulo, lema, weight=float(tiempo))
      GrafoAsoc.add_edge(estimulo, lema, weight=100-float(asociado))
  return GrafoFrec, GrafoTiempo, GrafoAsoc

In [7]:
GrafoFrec, GrafoTiempo, GrafoAsoc = build_graph(xlsx)

In [8]:
print(GrafoFrec.edges())

[('abeja', 'miel'), ('abeja', 'picar'), ('abeja', 'picadura'), ('abeja', 'reina'), ('abeja', '='), ('abeja', 'amarillo'), ('abeja', 'flor'), ('abeja', 'polen'), ('abeja', 'panal'), ('abeja', 'zumbido'), ('abeja', 'aguijón'), ('abeja', 'colmena'), ('abeja', 'volar'), ('abeja', 'dolor'), ('abeja', 'insecto'), ('abeja', 'animal'), ('abeja', 'avispa'), ('abeja', 'mosca'), ('abeja', 'raya'), ('abeja', 'alergia'), ('abeja', 'amarillo_negro'), ('abeja', 'bicho'), ('abeja', 'campo'), ('abeja', 'dulce'), ('abeja', 'hormiga'), ('abeja', 'jardín'), ('abeja', 'libar'), ('abeja', 'miope'), ('abeja', 'molestia'), ('abeja', 'natación'), ('abeja', 'pequeño'), ('abeja', 'roncha'), ('abeja', 'trabajador'), ('abeja', 'viaje'), ('abeja', 'viejo'), ('abeja', 'yo'), ('abeja', 'mariposa'), ('abeja', 'pañal'), ('abeja', 'plastilina'), ('abeja', 'reja'), ('miel', 'cacahuate'), ('miel', 'chupón'), ('miel', 'cuchara'), ('miel', 'cuna'), ('miel', 'dulce'), ('miel', 'hot_cakes'), ('miel', 'jabón'), ('miel', 'luna'

In [9]:
# Funcion que filtra stopwords y lematiza
def limpia_lematiza(cadena):
    limpiado = ""
    palabras_funcionales = nltk.corpus.stopwords.words('spanish')
    # Quita los signos de puntuacion
    for c in string.punctuation:
      cadena = cadena.replace(c," ")
    # Elimino espacios extra
    cadena = cadena.strip()
    # Itero cada palabra en la string
    for palabra in cadena.split():
      if palabra not in palabras_funcionales:
        # Lematizo
        doc = nlp(palabra)
        limpiado += doc[0].lemma_ + " "
    return limpiado.rstrip()


In [10]:
limpia_lematiza("insecto de rayas que produce miel")

'insecto rayas producir miel'

In [11]:
def conceptos(lista, subconjunto):
  datos = list()
  if len(lista)<= 100:
    tope = len(lista)
  else:
    tope = 100
  for i in range(0,tope):
    if str(lista[i][0]) not in subconjunto and float(lista[i][1])>0:
      datos.append(str(lista[i][0]))
  return datos

In [12]:
def diccionario_nap(definicion):
  GrafoFrec, GrafoTiempo, GrafoAsoc = build_graph(xlsx)
  texto = limpia_lematiza(definicion)
  texto = texto.lower()
  tokens = texto.split()
  subconjunto_lemas = list()
  for token in tokens:
    if token in GrafoFrec.nodes() and token != "":
      subconjunto_lemas.append(token)
  if(len(subconjunto_lemas)>0):
    # Grafo de Asociacion
    resultado_asociacion = nx.betweenness_centrality_subset(GrafoAsoc, subconjunto_lemas,
                                                            subconjunto_lemas, weight='weight', normalized=True)
    # Ordena con base en la centralidad (segundo valor de la tupla)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x:x[1], reverse=True)[0:100]
    print("Asociacion:", end=" ")
    print(conceptos(encontrado, subconjunto_lemas))

    # Grafo de Frecuencia
    resultado_asociacion = nx.betweenness_centrality_subset(GrafoFrec, subconjunto_lemas,
                                                            subconjunto_lemas, weight='weight', normalized=True)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x:x[1], reverse=True)[0:100]
    print("Frecuencia:", end=" ")
    print(conceptos(encontrado, subconjunto_lemas))

    # Grafo de Tiempo
    resultado_asociacion = nx.betweenness_centrality_subset(GrafoTiempo, subconjunto_lemas,
                                                            subconjunto_lemas, weight='weight', normalized=True)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x:x[1], reverse=True)[0:100]
    print("Tiempo:", end=" ")
    print(conceptos(encontrado, subconjunto_lemas))

  else:
    print("Definicion muy corta, agregar mas datos")
  return

In [13]:
diccionario_nap("animal conocido como el rey de la selva")

Asociacion: ['león']
Frecuencia: ['león']
Tiempo: ['león']


In [14]:
diccionario_nap("insecto de rayas que genera miel")

Asociacion: ['abeja']
Frecuencia: ['abeja', 'picar', 'araña']
Tiempo: ['abeja']


In [15]:
carpeta = "/content/drive/MyDrive/PLN_UADY/corpus definiciones"

In [16]:
for filename in os.listdir(carpeta):
  f = os.path.join(carpeta, filename)
  with open(f, "r") as file:
    lines = file.readlines()
    concepto = lines[0]
    print(f"Concepto Objetivo: \"{concepto[0:-1]}\"")
    for i in range(1, len(lines)):
      print(f"Definicion {i}: \"{lines[i].rstrip()}\"")
      diccionario_nap(lines[i].rstrip())
      print()


Concepto Objetivo: "Abrigo"
Definicion 1: "ropa como sueter que se usa para el frío pero más largo"
Asociacion: ['bufanda', 'vestir', 'pantalón', 'calzón', 'gorra']
Frecuencia: ['gorra', 'calor', 'cobija', 'sol', 'animal', 'caliente', 'cabeza', 'pelo', 'jirafa', 'plancha', 'vestir', 'short', 'pingüino']
Tiempo: ['vestir', 'suéter', 'pie', 'pantalón']

Definicion 2: "Lo que pones encima del sueter"
Asociacion: []
Frecuencia: []
Tiempo: []

Definicion 3: "Lo que usas para que ni te de frío"
Asociacion: ['agua', 'botella', 'muñeco_de_nieve']
Frecuencia: ['agua', 'botella', 'hielo', 'muñeco_de_nieve']
Tiempo: ['muñeco_de_nieve', 'botella', 'verde', 'agua']

Definicion 4: "ropa que sirve para protegerse del frío"
Asociacion: ['botella', 'suéter', 'botón', 'agua', 'leche', 'bufanda', 'jabón', 'lavadora']
Frecuencia: ['agua', 'botella', 'plato', 'guante', 'caliente', 'cobija', 'comida', 'plancha', 'refrigerador', 'hielo', 'vaso', 'lavabo']
Tiempo: ['botella', 'suéter', 'vaso', 'cocina', 'agua